### Comments:
*   DONE: Check year in publisher names, then update published years
*   DONE: Add title_length column
*   Compare performance with/without normalization
*   Feature extraction
*   Try XGBoost, SVM, CNN
*   Other models

# Import libraries

In [1]:
import json, re, ast
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import make_pipeline
from scipy.sparse import hstack, vstack

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

In [2]:
nltk.download('punkt')  # Download the Punkt tokenizer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anhlh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anhlh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load train data as a pandas dataframe

In [3]:
def to_dataframe(json_path, csv_name):
    # load json -> to csv -> to df
    df = pd.read_json(json_path)
    df.to_csv(csv_name, index=False)
    result = pd.read_csv(csv_name)
    # make all strings lowercase
    result= result.applymap(lambda s:s.lower() if type(s) == str else s)
    result['author'] = result['author'].fillna(value="[]")
    result['abstract'] = result['abstract'].fillna(value=" ")
    train = result.fillna(0)
    return train

## Change author to number of authors, then normalize it

In [4]:
scaler = MinMaxScaler()

def get_nr_authors(train):
  train['author'] = train['author'].apply(ast.literal_eval)
  train['author'] = train['author'].apply(len)

  nr_authors_2d = train['author'].values.reshape(-1, 1)
  train['author'] = scaler.fit_transform(nr_authors_2d)
  return train

 COMMENT: adding a column counting the length of the title of the publication.

In [5]:
def get_title_length(train):
  train['title_length'] = train['title'].apply(lambda x: len(x.split()))
  return train

## Encode entrytype

In [6]:
def encode_entrytype(train):
  # Get one ho t encoding of ENTRYTYPE column
  one_hot = pd.get_dummies(train['ENTRYTYPE'])
  one_hot = one_hot.astype(int)
  train = train.join(one_hot)
  train = train.drop('ENTRYTYPE', axis=1)
  return train

## Get the first and last years in which a publisher appeared

###### 1. Modify the inconsistent publisher names

In [7]:
inconsistency = ['atala', 'aslib', 'european language resources association', 'incoma', 'springer',
                'international committee on computational linguistics', 'linköping university electronic press',
                'northern european association for language technology', 'the korean society for language and information']


def rename_publishers(df, inconsistency):
    df.publisher = np.where((df.publisher.str.contains("not mentionned")),
                            "unknown", df.publisher)
    df.publisher = np.where((df.publisher.str.contains("association for computational linguistics"))& (~df.publisher.str.contains("and"))|
                            (df.publisher == "assocation for computational linguistics")|
                            (df.publisher == "association for computational lingustics"),
                            "association for computational linguistics", df.publisher)
    df.publisher = np.where((df.publisher.str.contains("northern european association"))&
                             (df.publisher.str.contains("language technology")),
                             "northern european association for language technology", df.publisher)
    df.publisher = np.where((df.publisher.str.contains("international committee"))&
                             (df.publisher.str.contains("computational linguistics")),
                             "international committee for computational linguistics", df.publisher)
    df.publisher = np.where((df.publisher.str.contains("european language")),
                             "european language resources association", df.publisher)

    for incon in inconsistency:
        df.publisher = np.where(df.publisher.str.contains(incon), incon, df.publisher)

    df = df[df["publisher"] != "unknown"]
    return df

###### 2. Get the first and last years in which a publisher appeared


In [8]:
years_in_names = list(range(1962, 2024))

def manual_update_years(pub_agg, years_in_names):
  """Updates the first and last years if the name of the publisher contains a year"""
  for year in years_in_names:
    pub_agg['first_published'] = np.where(
                                          (pub_agg.publisher.str.contains(str(year))),
                                          year,
                                          pub_agg['first_published']
                                        )
    pub_agg['last_published'] = np.where(
                                          (pub_agg.publisher.str.contains(str(year))),
                                          year,
                                          pub_agg['last_published']
                                        )
  return pub_agg

In [9]:
scaler = StandardScaler()


def publisher_years(train, inconsistency):
  train = rename_publishers(train, inconsistency)
  pub = train[["publisher", "year"]]
  pub_agg = pub.groupby("publisher").agg(first_published=('year','min'),
                                        last_published=('year','max')).reset_index()
  pub_agg = manual_update_years(pub_agg, years_in_names)
  pub_agg[['first_published', 'last_published']] = scaler.fit_transform(pub_agg[['first_published', 'last_published']])
  return pub_agg


def join_published_years(train, pub_agg):
  #Join the pub_agg with train
  train_processed = pd.merge(train, pub_agg, on='publisher', how='right')
  train_processed = train_processed.drop('publisher', axis=1)
  return train_processed

In [10]:
##check the specfic publishers if you want:
# pub_agg.publisher.unique()

In [11]:
##check the publishers that are long gone:
# pub_agg.sort_values(by=["last_published"], ascending=True).head(15)

## Extract information from 'title' column
### 1.1 Tokenize the title

In [12]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


#Tokenization function
def tokenize_text(text):
    tokens = word_tokenize(text)
    new_tokens = []
    for word in tokens:
        #Remove special characters and stop words in the tokens
        word = re.sub(r'[^A-Za-z\s]', '', word)
        if word not in stop_words and word != '':
            # Reduce words to their root form to handle variations
            word = stemmer.stem(word)
            new_tokens.append(word)
    tokenized_title = ' '.join(new_tokens)
    return tokenized_title


#Apply tokenization to the 'title' column
#Notice that the original title column is overwritten.
###For saving time you may check the first 100 rows before applying to the whloe dataframe:
#train = train.loc[0:100]



## 1.2 Tokenize the abstract

### 2. Stemming by Vectorizer


In [13]:
# Generator function to yield batches of text
def batch_generator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# HashingVectorizer with incremental learning
vectorizer = HashingVectorizer(n_features=1000, alternate_sign=False)

# Process data in batches
batch_size = 2

In [14]:
def stem_title(train):
  sparse_data_list = []
  for batch in batch_generator(train['title'], batch_size):
      X_batch = vectorizer.transform(batch)
      sparse_data_list.append(X_batch)

  # Concatenate the sparse matrices vertically
  X_sparse = vstack(sparse_data_list, format='csr')

  # Convert the sparse matrix to a DataFrame
  vectorized_df = pd.DataFrame(X_sparse.toarray(), columns=[f'feature_{i}' for i in range(X_sparse.shape[1])])

  # Concatenate the original DataFrame and the vectorized DataFrame horizontally
  train = pd.concat([train, vectorized_df], axis=1)
  train = train.drop('title', axis=1)
  return train

In [15]:
def stem_abstract(train):
  sparse_data_list_abs = []
  for batch in batch_generator(train['abstract'], batch_size):
      abs_batch = vectorizer.transform(batch)
      sparse_data_list_abs.append(abs_batch)

  # Concatenate the sparse matrices vertically
  abs_sparse = vstack(sparse_data_list_abs, format='csr')

  # Convert the sparse matrix to a DataFrame
  vectorized_df = pd.DataFrame(abs_sparse.toarray(), columns=[f'feature_{i}' for i in range(abs_sparse.shape[1])])

  # Concatenate the original DataFrame and the vectorized DataFrame horizontally
  train = pd.concat([train, vectorized_df], axis=1)
  train = train.drop('abstract', axis=1)
  return train

# Load train and test, then pre-process them

In [16]:
def pre_process(train, pub_agg):
  train = train.drop(['editor'], axis=1)
  train = get_nr_authors(train)
  train = get_title_length(train)
  train = encode_entrytype(train)
  train = rename_publishers(train, inconsistency)
  train = join_published_years(train, pub_agg)
  train['title'] = train['title'].apply(tokenize_text)
  train['title'] = train['title'].fillna(value="")
  train['abstract'] = train['abstract'].apply(tokenize_text)
  train['abstract'] = train['abstract'].fillna(value="")
  train = stem_title(train)
  train = stem_abstract(train)
  return train

In [17]:
train = to_dataframe('train.json', 'train.csv')
test = to_dataframe('test.json', 'test.csv')
pub_agg = publisher_years(train, inconsistency)

In [18]:
train = pre_process(train, pub_agg)
# print(train.dtypes)
test = pre_process(test, pub_agg)
# print(test.dtypes)

In [19]:
X = train.drop('year', axis=1)
y = train['year'].values

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=11)

In [21]:
print(test.head())

     author  title_length  article  inproceedings  proceedings  \
0  0.351939            12        0              1            0   
1  0.351939            12        0              1            0   
2  0.351939            15        0              1            0   
3  2.965848            20        0              1            0   
4 -1.390666            20        0              0            1   

   first_published  last_published  feature_0  feature_1  feature_2  ...  \
0         1.125159        1.038567        0.0        0.0        0.0  ...   
1         1.125159        1.038567        0.0        0.0        0.0  ...   
2         1.125159        1.038567        0.0        0.0        0.0  ...   
3         1.125159        1.038567        0.0        0.0        0.0  ...   
4         1.125159        1.038567        0.0        0.0        0.0  ...   

   feature_990  feature_991  feature_992  feature_993  feature_994  \
0          0.0          0.0          0.0          0.0          0.0   
1     

# Regression / Multi-class Classification in Neural Network

Implementation for Softmax Regression:
1. Basic
2. Use a linear activation function in the output layer.


## MLPRegressor
### MAE: 3.952511946715513

**The model below takes approximately 15 min to run.**

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50),
                   max_iter=1000,
                   random_state=11,
                   activation='relu',   # Use ReLU activation in hidden layers
                   solver='adam',       # Use the Adam optimizer
                   alpha=0.0001,        # L2 regularization term
                   )
mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=11)

In [ ]:
# Make predictions on the validation set
y_val_pred_mlp = mlp.predict(X_val)
# Evaluate the model on the validation set
mae_val = mean_absolute_error(y_val, y_val_pred_mlp)
mae_val

3.952511946715513

## MLPClassfier
### MAE: 3.5305892547660314

**The model below took around 10min**
But it took 42min using uni wifi.

In [ ]:
mlp_classifier = MLPClassifier(
                                hidden_layer_sizes=(50, 25),
                                max_iter=500,
                                random_state=11,
                                activation='logistic' # 'logistic' activates the softmax function
                              )

In [ ]:
mlp_classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 25), max_iter=500,
              random_state=11)

In [ ]:
# Make predictions on the validation set
y_val_pred_mlp_c = mlp_classifier.predict(X_val)
# Calculate MAE on the validation set
mae_val = mean_absolute_error(y_val, y_val_pred_mlp_c)
print(mae_val) #3.5305892547660314
print(y_val[0:10])
print(y_val_pred_mlp_c[0:10])

3.5305892547660314
[2020 2016 2023 2022 2021 2011 2023 2021 2022 2012]
[2021 2008 2017 2021 2017 2008 2019 2020 2020 2001]


# Regression / Multi-class Classification in Decision Trees

##DecisionTree Regressor
###MAE: 3.4001201668941414


took 27min to run

In [ ]:
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dtr = DecisionTreeRegressor(random_state=11)

# Use GridSearchCV to search through the parameter grid
grid_search = GridSearchCV(dtr, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Evaluate the model with the best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error on Validation Set: {mae}")

KeyboardInterrupt: ignored

## DecisionTreeClassifier
### MAE: 3.787175043327556


In [ ]:
dtc = DecisionTreeClassifier(
                              random_state=11,

                            )      # Make an instance of the Model
dtc.fit(X_train, y_train)           # Training the model on the data (train set)

In [ ]:
# Make predictions on the validation set
y_val_pred_dtc = dtc.predict(X_val)
# Evaluate the model on the validation set
mae_val = mean_absolute_error(y_val, y_val_pred_dtc)
mae_val

In [ ]:
# Perform 5-fold cross-validation
cross_val_scores = cross_val_score(dtc, X, y, cv=5, scoring='neg_mean_absolute_error')

# Display the cross-validation scores
print("Cross-Validation Scores:", -cross_val_scores)  # Negate scores to get positive MAE values
print("Mean MAE:", -cross_val_scores.mean())           # Calculate mean MAE

# Regression / Multi-class Classification in Random Forest

## RandomForestRegressor
### MAE: 2.987849547676082

**The model below takes approximately 57 min to run!!**

In [ ]:
rfr = RandomForestRegressor(random_state=11)      # Make an instance of the Model
rfr.fit(X_train, y_train)           # Training the model on the data (train set)

y_val_pred_rfr = rfr.predict(X_val)
mae_score = mean_absolute_error(y_val, y_val_pred_rfr)
print('MAE on validation set: ', mae_score)

##RandomForestRegressor with tuning
###MAE: 2.987849547676082

In [ ]:
rf_params = {
    'n_estimators': 100,      # Number of trees in the forest
    'max_depth': None,        # Maximum depth of the tree
    'min_samples_split': 2,   # Minimum number of samples required to split an internal node
    'min_samples_leaf': 1,    # Minimum number of samples required to be at a leaf node
    'random_state':11
}

# Create an instance of the model with specified hyperparameters
rf = RandomForestRegressor(**rf_params)
rf.fit(X_train, y_train)           # Training the model on the data (train set)

y_val_pred_rf = rf.predict(X_val)
mae_score = mean_absolute_error(y_val, y_val_pred_rf)
print('MAE on validation set: ', mae_score)

##RandomForestRegressor with tuning 2.0
###MAE:

In [ ]:
param_grid = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestRegressor(random_state=11)

# Use GridSearchCV to search through the parameter grid
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Evaluate the model with the best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
print(f"Mean Absolute Error on Validation Set: {mae}")

In [ ]:
print(y_val[0:50])
print(y_val_pred_dtc[0:50])

# Experiment Session

In [ ]:
model = Sequential([
    Dense(units=100, activation='relu'),
    Dense(units=50, activation='relu'),
    Dense(units=1, activation='softmax')  # Linear activation for regression
], name="basic")

model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(train_clean.drop('year', axis=1), train_clean['year'].values, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
1443/1443 [==============================] - 9s 6ms/step - loss: 2013.9005 - val_loss: 2012.6420
Epoch 2/50
1443/1443 [==============================] - 6s 4ms/step - loss: 2013.9014 - val_loss: 2012.6420
Epoch 3/50
1443/1443 [==============================] - 8s 6ms/step - loss: 2013.8992 - val_loss: 2012.6420
Epoch 4/50
1443/1443 [==============================] - 6s 4ms/step - loss: 2013.8993 - val_loss: 2012.6420
Epoch 5/50
1443/1443 [==============================] - 10s 7ms/step - loss: 2013.9021 - val_loss: 2012.6420
Epoch 6/50
1443/1443 [==============================] - 8s 5ms/step - loss: 2013.8998 - val_loss: 2012.6420
Epoch 7/50
1443/1443 [==============================] - 7s 5ms/step - loss: 2013.9014 - val_loss: 2012.6420
Epoch 8/50
1443/1443 [==============================] - 8s 5ms/step - loss: 2013.9000 - val_loss: 2012.6420
Epoch 9/50
1443/1443 [==============================] - 7s 5ms/step - loss: 2013.9006 - val_loss: 2012.6420
Epoch 10/50
1443/1443 [====

In [ ]:
train_predictions = model.predict(train_clean.drop('year', axis=1))
mae_train = tf.keras.losses.mean_absolute_error(train_clean['year'].values, train_predictions)


1803/1803 [==============================] - 4s 2ms/step


In [ ]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [
        ### START CODE HERE ###
        Dense(units=128, activation='relu'),
        Dense(units=64, activation='relu'),
        Dense(units=50, activation='softmax')

        ### END CODE HERE ###
    ], name = "basic"
)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy())

In [ ]:
model.fit(train_clean.drop('year', axis=1), train_clean['year'].values)

InvalidArgumentError: ignored

In [ ]:
len(train.editor.unique())

In [ ]:
# The paper with the most authors
from ast import literal_eval
maximum = 0
total_authors = sorted_train.author.unique()

for author in total_authors:
    if type(author) is str:
        author = literal_eval(author)
        if len(author) > maximum:
            maximum = len(author)
            longest_name = author

print(maximum, longest_name)


In [ ]:
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

dummy.fit(train.drop('year', axis=1), train['year'].values)

In [ ]:
# from sklearn.feature_extraction.text import HashingVectorizer
# import pandas as pd

# # Vectorization using HashingVectorizer
# vectorizer = HashingVectorizer()  # Adjust n_features as needed
# X = vectorizer.transform(train_processed['title'])

# # Convert the sparse matrix to a DataFrame
# count_df = pd.DataFrame(X.toarray(), columns=[f'feature_{i}' for i in range(X.shape[1])])

# # Concatenate the HashingVectorizer DataFrame with the original DataFrame
# # train_processed = pd.concat([train_processed, count_df], axis=1)

In [ ]:
# # Vectorization
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(train_processed['title'])
# count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# # Concatenate the TF-IDF DataFrame with the original DataFrame
# train_processed= pd.concat([train_processed, count_df], axis=1)

In [ ]:
# train_processed = pd.concat([train_processed, count_df], axis=1)

In [ ]:
featurizer = ColumnTransformer(transformers=[("title", CountVectorizer(), "title")])
featurizer.fit(train_processed[['title']])

transformed_title = featurizer.transform(train_processed[['title']])
transformed_title.shape


(57696, 27134)

In [ ]:
# df_transformed_title = pd.DataFrame(transformed_title)


In [ ]:
train_clean.dtypes

year               int64
author           float64
abstract          object
article            int64
inproceedings      int64
                  ...   
feature_995      float64
feature_996      float64
feature_997      float64
feature_998      float64
feature_999      float64
Length: 1008, dtype: object

# XGBoost

In [27]:
pip install xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/bc/43/242432efc3f60052a4a534dc4926b21e236ab4ec8d4920c593da3f65c65d/xgboost-2.0.2-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 653.6 kB/s eta 0:02:33
   ---------------------------------------- 0.3/99.8 MB 3.3 MB/s eta 0:00:31
   ---------------------------------------- 0.9/99.8 MB 6.4 MB/s eta 0:00:16
   ---------------------------------------- 1.2/99.8 MB 6.3 MB/s eta 0:00:16
    --------------------------------------- 1.9/99.8 MB 8.1 MB/s eta 0:00:13
    --------------------------------------- 2.2/99.8 MB 8.3 MB/s eta 0:00:12
   - -------------------------------------- 2.9/99.8 MB 8.8 MB/s eta 0:00:12
   - -------------------------------------- 3.3/99.8 MB 9.5 MB/s eta 0:00:11
   - -------------------------------

In [28]:
#pip install xgboost
#conda install -c conda-forge xgboost

import xgboost as xgb  # Import the XGBoost library

xgb_model = xgb.XGBClassifier(  # Instantiate the XGBoost model
    max_depth=3,  # Set the maximum depth of each tree to 3
    learning_rate=0.1,  # Set the learning rate to 0.1
    n_estimators=100,  # Use 100 trees in the ensemble
    objective='binary:logistic',  # Use the logistic regression objective for binary classification
    random_state=11  # Set a random seed for reproducibility
)

xgb_model.fit(X_train, y_train)  # Train the XGBoost model using the training data

y_pred_xgb = xgb_model.predict(X_val)  # Make predictions on the test data using the trained model

mae_xgb = mean_absolute_error(y_val, y_pred_xgb)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47], got [1962 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989
 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020 2021 2022 2023]

# SVM

In [29]:
from sklearn import svm  # Import the SVM module from scikit-learn

# Step 3: Instantiate the SVM model
svm_model = svm.SVC(kernel='linear', C=1.0, random_state=11)
# Create an instance of the SVM model with a linear kernel, regularization parameter C of 1.0, and random state for 
# reproducibility

# Step 4: Train the model
svm_model.fit(X_train, y_train)

# Step 5: Make predictions
y_pred_svm = svm_model.predict(X_val)
# Use the trained SVM model to predict the labels for the validation data

mae_svm = mean_absolute_error(y_val, y_pred_svm)
mae_svm

3.494887348353553

# CNN

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

image_width = 64
image_height = 64
num_channels = 3

# Step 3: Define the CNN architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_width, image_height, num_channels)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
# Create a sequential model with convolutional, pooling, flattening, and fully connected layers

# Step 4: Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Configure the model with an optimizer, loss function, and additional metrics

# Step 5: Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)
# Fit the model to the training data (X_train, y_train) for a specified number of epochs and batch size

# Step 6: Evaluate the model
y_pred_cnn = model.predict(X_val)
# Use the trained SVM model to predict the labels for the validation data

mae_cnn = mean_absolute_error(y_val, y_pred_cnn)
mae_cnn

NameError: name 'num_classes' is not defined